In [ ]:
# importing libraries
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torchvision.utils import make_grid

"""We'll download the images in PNG format from [this page](https://course.fast.ai/datasets), using some helper functions from the `torchvision` and `tarfile` packages."""
#function :  x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z)):https://www.desmos.com/calculator/0hiyjyun2f (move points to play with the parameters)
#what number the paramter start and end
fromI=1
toI=100

fromL=0.1
toL=1000

fromQ=0.1
toQ=100

LogarithmINC=True#insted of incresing when checking parameters with a constant , will will get to the to value useing geometric progression

#size of splits will, example fromI=1 toI=3,SIZEI=3 then i will check 1,2,3
SIZEI=0#how many i to check
SIZEL=0#how many l to check
SIZEQ=0#how many q to check

checkCertainParams=False
#if the one above is true set the parameters u want to check else it will igonre the value
L=46.41588833612777#0.1->2
Q=0.1 #0.01->0.2
P=2 #NO CHANGE
Z=10# NO CHANGE
I=21.544346900318832 #1->3



train_running_size=500#train size of data
val_running_Size=1000#val size of data
setAll=False# will train and predict useing all the data ignoring the 2 above
batch_size = 256 #hyper parameter

num_epochs = 128
opt_func = torch.optim.Adam
lr = 0.001
random_seed = 10
val_size = 5000

In [ ]:
if(checkCertainParams):
    L0=L
    Q0=Q
    I0=I
    Z0=Z
    P0=P

def dynamic(input):
    x=input
    x[x<0]=0
    if (checkCertainParams):
        x = x + (L0 / (Q0 * (x + I0) ** P0 + Z0)) - (L / (Q0 * (x - I0) ** P0 + Z0))
    else:
        x = x + (L / (Q * (x + I) ** P + Z)) - (L / (Q * (x - I) ** P + Z))
    return x

class Dynamic(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return dynamic(input) # simply apply already implemented SiLU

def relu(input):
    x=input
    x[x<0]=0
    return x

class RELU(nn.Module):
    def __init__(self):
        super().__init__() # init the base class
    def forward(self, input):
        return relu(input) # simply apply already implemented SiLU

def setParams(L_,Q_,P_,Z_,I_):
    global L
    global Q
    global P
    global Z
    global I
    L = L_
    Q = Q_
    P = P_
    Z = Z_
    I = I_

    pass

activation_function = RELU()
print("activation_function set")



In [ ]:
"""### Loading and Processing Dataset"""

from torchvision.datasets.utils import download_url

# Download the dataset
dataset_url = 'https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz'
download_url(dataset_url, '.')

#Extract from archive
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
  tar.extractall(path='./data')

# Look into the data directory
data_dir = './data/cifar10'
classes = os.listdir(data_dir + "/train")


#Data transforms (normalization and data augmentation)

stats = ((0.4914, 0.4822, 0.4465),
         (0.2023, 0.1994, 0.2010))

train_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'),
                         tt.RandomHorizontalFlip(),
                         tt.ToTensor(),
                         tt.Normalize(*stats, inplace=True)])

valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])

# PyTorch datasets
train_ds = ImageFolder(data_dir+'/train', train_tfms)
valid_ds = ImageFolder(data_dir+'/test', valid_tfms)
if(setAll):
    train_running_size=len(train_ds)
    val_running_Size=len(valid_ds)
train_ds, empty=random_split(train_ds, [train_running_size, len(train_ds)-train_running_size])
valid_ds, empty=random_split(valid_ds,[val_running_Size,len(valid_ds)-val_running_Size])

# Pytorch Data Loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size*2, num_workers=2, pin_memory=True)

"""### Uploading on GPU"""

### Using a GPU
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)


In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

"""Building our architecture:"""

def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              activation_function]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out



"""### Training the Model
The improvements in fit functions are:
1. Learning rate scheduling: Instead of using a fixed learning rate, we will use a learning rate scheduler, which will change the learning rate after every batch of training. We will use one cycle policy [1cycle policy](https://sgugger.github.io/the-1cycle-policy.html).
2. Weight Decay: A regularization technique which prevents the weights from becoming too large by adding an additional term to the loss function.
3. Gradient clipping: Apart from the layer weights and outputs, it also helpful to limit the values of gradients to a small range to prevent undesirable changes in parameters due to large gradient values

"""

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader,
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs,
                                                steps_per_epoch=len(train_loader))

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

# Evaluating the model before training



In [ ]:
epochs = num_epochs
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam


print("RELU on resnet9:")
activation_function=RELU()
model = to_device(ResNet9(3, 100), device)
model
history = [evaluate(model, valid_dl)]
history

# Commented out IPython magic to ensure Python compatibility.
# %%time

tic = time.time()
history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl,
                             grad_clip=grad_clip,
                             weight_decay=weight_decay,
                             opt_func=opt_func)
toc = time.time()
print("time took:", toc - tic)

print("founded function on resnet9:")
activation_function=Dynamic()
model = to_device(ResNet9(3, 100), device)
model
history = [evaluate(model, valid_dl)]
history


bestScore=0
bestI=0
bestQ=0
bestL=0


activation_function = Dynamic()
I_=0
Q_=0
L_=0
for i in range(SIZEI):
    for l in range(SIZEL):
        for q in range(SIZEQ):
            model = to_device(ResNet9(3, 10), device)
            to_device(model, device)
            history = [evaluate(model, valid_dl)]

            I_=fromI+i*(toI-fromI)/SIZEI
            Q_=fromQ+q*(toQ-fromQ)/SIZEQ
            L_=fromL+l*(toL-fromL)/SIZEL

            if(LogarithmINC):
                I_=(((toI/fromI)**(1/SIZEI))**i)*fromI
                Q_=(((toQ/fromQ)**(1/SIZEQ))**q)*fromQ
                L_=(((toL/fromL)**(1/SIZEL))**l)*fromL
                pass
            setParams(L_,Q_,P,Z,I_)
            tic = time.time()
            history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl,
                                     grad_clip=grad_clip,
                                     weight_decay=weight_decay,
                                     opt_func=opt_func)
            toc = time.time()
            print("time took:", toc - tic)
            score=history[len(history)-1]["val_acc"]
            if score>bestScore:
                bestI = I_
                bestQ = Q_
                bestL = L_
                bestScore=score
            print(score,"with i=",I_,"q=",Q_,"l=",L_)

if not checkCertainParams:
  print("best score:",bestScore,"with i=",I_,"q=",Q_,"l=",L_)